* Which year was the largest production ever made, narrow it down to the month with the highest?
* Which year has the highest average production for our records, and which year has the highest average price.
* Is there a month where the total production was exported totally?
* How many times was oil exported less than 70% of the production.

View of production and prices in three years difference

What is the correlation of price with production and export?

Summarize for the last five years, the average price, standard deviation and median. 

In [26]:
import sqlite3 as sq

In [27]:
conn = sq.connect("crude_oil.db")

In [28]:
# Create table called crudeOilPrice
query = """
CREATE TABLE crudeOilPrice(
    year VARCHAR(4),
    month SMALLINT,
    crude_oil_price DECIMAL(5,2),
    production DECIMAL(3,2),
    crude_oil_export DECIMAL(3,2)
    )"""

with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS crudeOilPrice")
    cur.execute(query)

In [29]:
# checking our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
print(cur.fetchall())

[('crudeOilPrice',)]


In [30]:
# previewing the table content
query = "SELECT * FROM crudeOilPrice"
cur.execute(query)
print(cur.fetchall())

[]


In [31]:
# coverting sql table to dataframe
import pandas as pd

sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export


Let fill in values from the csv


In [32]:
# Loading CSV file
csv = pd.read_csv('CrudeOilProdPrice25102022.csv', index_col=False)
csv.head()

,Year,Month,Crude Oil Price,Production,Crude Oil Export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83


In [33]:
# creating a tuple of values in each row
csvTuple = tuple(csv.itertuples(index=False, name=None))
csvTuple[0]

(2006, 1, 63.85, 2.59, 2.14)

In [34]:
# inserting into table on the database
insertQuery = "INSERT INTO crudeOilPrice values(?,?,?,?,?)"
cur.executemany(insertQuery, csvTuple)

In [35]:
# Previewing table
query = "SELECT * FROM crudeOilPrice"

# Reading table as a pandas dataframe
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,year,month,crude_oil_price,production,crude_oil_export
0,2006,1,63.85,2.59,2.14
1,2006,2,61.33,2.47,2.02
2,2006,3,65.00,2.25,1.80
3,2006,4,72.09,2.32,1.87
4,2006,5,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,5,116.72,1.02,0.57
197,2022,6,130.10,1.16,0.71
198,2022,7,120.54,1.08,0.63
199,2022,8,106.34,0.97,0.52


## Data Cleaning

In [36]:

query = """
UPDATE
    crudeOilPrice
SET 
    month = 
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(
        REPLACE(month, 12, 'December'), 
        2, 'February'), 
        3, 'March'),
        4, 'April'),
        5, 'May'), 
        6, 'June'),
        7, 'July'),
        8, 'August'),
        9, 'September'),
        10, 'October'),
        11, 'November'),
        1, 'January')
"""

cur.execute(query)

In [37]:
query = "SELECT * FROM crudeOilPrice"
pd.read_sql_query(query, conn)

,year,month,crude_oil_price,production,crude_oil_export
0,2006,January,63.85,2.59,2.14
1,2006,February,61.33,2.47,2.02
2,2006,March,65.00,2.25,1.80
3,2006,April,72.09,2.32,1.87
4,2006,May,71.18,2.28,1.83
...,...,...,...,...,...
196,2022,May,116.72,1.02,0.57
197,2022,June,130.10,1.16,0.71
198,2022,July,120.54,1.08,0.63
199,2022,August,106.34,0.97,0.52


In [38]:
query = """
SELECT DISTINCT month FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,month
0,January
1,February
2,March
3,April
4,May
5,June
6,July
7,August
8,September
9,October


In [39]:
# Overview of record
query = """
SELECT COUNT(DISTINCT year) AS Years_record,
       MIN(year) AS Start_year,
       MAX(year) AS End_year
  FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,Years_record,Start_year,End_year
0,17,2006,2022


* Which year was the largest production ever made, narrow it down to the month?

In [40]:
query = """
SELECT year, month, max(production) AS Max_Prod
FROM crudeOilPrice
"""
pd.read_sql_query(query, conn)

,year,month,Max_Prod
0,2010,October,2.88


* Which year has the highest average production for our record.


In [41]:
query = """
SELECT sub.Year, max(sub.Avg_Prod) AS Max_Avg_Prod
FROM(
SELECT year AS Year, avg(production) AS Avg_Prod
FROM crudeOilPrice
GROUP BY Year) AS sub
"""
pd.read_sql_query(query, conn)

,Year,Max_Avg_Prod
0,2010,2.4675


 * Which year has the highest average price.

In [42]:
query = """
SELECT sub.Year, max(Avg_Price) AS Max_Avg_Price
FROM(
SELECT year AS Year, avg(crude_oil_price) AS Avg_Price
FROM crudeOilPrice
GROUP BY Year) AS sub
"""
pd.read_sql_query(query, conn)

,Year,Max_Avg_Price
0,2011,113.76


Is there a month where the total production was exported totally?

In [43]:
query = """
SELECT COUNT(*) AS count
FROM crudeOilPrice
WHERE production = crude_oil_export
"""
pd.read_sql_query(query, conn)

,count
0,0


How many times was oil exported less than 70% of the production.

In [44]:
query = """
SELECT COUNT(*) AS Count
FROM (
SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
FROM crudeOilPrice)
WHERE crude_oil_export < fifty_perc_prod
"""
pd.read_sql_query(query, conn)

,Count
0,22


We have 22 count, let's narrow it down to year with the highest count.

In [45]:
query = """
SELECT year AS Year, COUNT(*) AS Export_Count
FROM (
SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
FROM crudeOilPrice)
WHERE crude_oil_export < fifty_perc_prod
GROUP BY Year
"""
pd.read_sql_query(query, conn)

,Year,Export_Count
0,2020,1
1,2021,12
2,2022,9


In [46]:
query = """
SELECT production, year, crude_oil_export, 0.7 * production AS fifty_perc_prod
FROM crudeOilPrice
LIMIT 10
"""
pd.read_sql_query(query, conn)

,production,year,crude_oil_export,fifty_perc_prod
0,2.59,2006,2.14,1.813
1,2.47,2006,2.02,1.729
2,2.25,2006,1.80,1.575
3,2.32,2006,1.87,1.624
4,2.28,2006,1.83,1.596
5,2.41,2006,1.96,1.687
6,2.39,2006,1.94,1.673
7,2.40,2006,1.95,1.680
8,2.40,2006,1.95,1.680
9,2.38,2006,1.93,1.666


We have seen the year that exported oil was below 70% of production. 

Let's look at when exported oil was above 80%

In [47]:
query = """
SELECT year AS Year, COUNT(*) AS Export_Count
FROM (
SELECT production, year, crude_oil_export, 0.8 * production AS fifty_perc_prod
FROM crudeOilPrice)
WHERE crude_oil_export >= fifty_perc_prod
GROUP BY Year
"""
pd.read_sql_query(query, conn)

,Year,Export_Count
0,2006,12
1,2007,5
2,2008,1
3,2009,1
4,2010,11
5,2011,12
6,2012,8
7,2013,2
8,2014,3


It turns out that the last time oil was exported above 80% of production was in 2014. And its only in 2006 and 2011 that 80% of produced oil was exported in all months.

In [48]:
query = """
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2011
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 5
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,month,Price2006,Price2011
0,January,63.85,97.96
1,February,61.33,106.57
2,March,65.00,116.56
3,April,72.09,124.49
4,May,71.18,118.43
5,June,69.32,117.03
6,July,75.13,117.86
7,August,75.15,111.99
8,September,62.97,115.73
9,October,59.49,113.12


View of price in three years gap

In [49]:
query = """
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2011
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 3 
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

,month,Price2006,Price2011
0,January,63.85,44.95
1,February,61.33,46.52
2,March,65.00,49.70
3,April,72.09,51.16
4,May,71.18,60.02
5,June,69.32,72.24
6,July,75.13,66.52
7,August,75.15,74.00
8,September,62.97,70.22
9,October,59.49,78.25


In [ ]:
query = """
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2011
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 3 
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

In [50]:
frr

NameError: name 'frr' is not defined

In [ ]:
query = """
SELECT year AS Year, avg(production) AS Avg_Prod
FROM crudeOilPrice
GROUP BY Year
"""
pd.read_sql_query(query, conn)

In [ ]:
query = """
SELECT AVG(crude_oil_price) AS average_oil_price
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

In [ ]:
query = """
SELECT AVG(production) AS average_production
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

In [ ]:
query = """
SELECT SUM(crude_oil_export) AS total_crude_oil_exported
FROM crudeOilPrice
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

In [ ]:
month_order = ['January', 'February', 'March', 'April', 
               'May', 'June', 'July', 'August', 'September']

query = """
SELECT  month, production AS "total_production_(mbd)", 
        crude_oil_price AS "total_price_(US$/Barrel)"
FROM crudeOilPrice
WHERE year = '2022'
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
sqlDF

In [ ]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
WHERE year = '2022'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

In [ ]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
WHERE year = '2021'
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

In [ ]:
query = """
SELECT month, SUM(production) AS total_production, year
FROM crudeOilPrice
GROUP BY month
"""
sqlDF = pd.read_sql_query(query, conn)
print(sqlDF)

In [ ]:
import plotly.graph_objs as go

In [ ]:
# Filtering monthly production
query = """
SELECT c1.month, SUM(c1.production) AS total_prod_2021, sub.total_prod_2022
FROM crudeOilPrice AS c1,
                        (SELECT month, SUM(production) AS total_prod_2022
                        FROM crudeOilPrice
                        WHERE year = '2022'
                        GROUP BY month) AS sub
WHERE year = '2021' AND (c1.month = sub.month) 
GROUP BY c1.month
"""
sqlDF = pd.read_sql_query(query, conn, index_col='month').reindex(month_order)
print(sqlDF)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2021,
                         name='2021 Production', line=dict(color='firebrick', 
                                                           width=4, dash='dot')))
fig.add_trace(go.Scatter(x=sqlDF.index, y=sqlDF.total_prod_2022,
                         name='2022 Production', line=dict(color='royalblue', 
                                                           width=4, dash='dot')))
fig.update_layout(title='Crude Oil Production (mbd)',
                  yaxis_title = 'Millions Barrels Per Day (mbd)',
                  )
fig.show()

In [ ]:
query = """
    SELECT  c1.month, c1.crude_oil_price AS Price2006, c2.crude_oil_price AS Price2011
      FROM  crudeOilPrice AS c1
INNER JOIN  crudeOilPrice AS c2
        ON  c1.month = c2.month
     WHERE  c1.year = 2006 AND c2.year = c1.year + 5
"""
sqlDF = pd.read_sql_query(query, conn)
sqlDF

In [ ]:
jupyter nbconvert Crude_oil_analysis.ipynb --to slides --post serve --no-input --no-prompt